In [6]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rio

# from uavsar_pytools.convert.tiff_conversion import read_annotation 

import matplotlib.pyplot as plt

from pathlib import Path

from tqdm import tqdm

# for site in np.unique([k.split('_')[0] for k in uavsars.keys()]):
#     if site != 'stlake': continue
#     print(site)
#     concat_dss = []
    
#     headings = {}
#     for i, (fp, ds) in tqdm(enumerate(dss.items()), total = len(dss)):

#         ann = pd.read_csv(list(fp.parent.glob('*.csv'))[0], index_col = 0)
        
#         t1, t2 = ann.loc['value', 'start time of acquisition for pass 1'], ann.loc['value', 'start time of acquisition for pass 2']
#         # round to day to match heading dates
#         t1, t2 = pd.to_datetime(t1).round('d'), pd.to_datetime(t2).round('d')
#         if t1.year != 2021: continue
#         # heading = ann.loc['value', 'peg heading']
#         heading = int(fp.stem.split('_')[1][:3])
#         if heading not in headings: headings[heading] = fp

#         pol = fp.stem.split('_')[-2][4:]

In [35]:
fig_dir = Path('/bsuhome/zacharykeskinen/uavsar-coherence/figures')
out_dir = Path('/bsuhome/zacharykeskinen/scratch/coherence/uavsar/gis')
tiff_dir = Path('/bsuhome/zacharykeskinen/scratch/coherence/uavsar/tiffs')

site = 'stlake'
fps = [fp for fp in list(tiff_dir.glob('*')) if site in fp.stem]

for d in tqdm(fps):
    ann = pd.read_csv(list(d.glob('*.csv'))[0], index_col = 0)
    t1, t2 = ann.loc['value', 'start time of acquisition for pass 1'], ann.loc['value', 'start time of acquisition for pass 2']
    t1, t2 = pd.to_datetime(t1).round('d'), pd.to_datetime(t2).round('d')
    if t1.year != 2021: continue
    # heading = ann.loc['value', 'peg heading']
    heading = d.stem.split('_')[1][:3]
    if heading != '091': continue

    concats = []
    for fp in d.glob('*.cor.grd.tiff'):
        pol = fp.stem.split('_')[-2][4:]
        da = xr.open_dataarray(fp).squeeze('band', drop = True).expand_dims(pol = [pol])
        if len(concats) != 0: da = da.rio.reproject_match(concats[0])
        concats.append(da)
    ds = xr.concat(concats, 'pol').sel(pol = ['VV', 'VH', 'HH'])
    ds.rio.to_raster(out_dir.joinpath(f'{pd.to_datetime(t1).date()}_{pd.to_datetime(t2).date()}.tif'))

  0%|          | 0/21 [00:00<?, ?it/s]

100%|██████████| 21/21 [01:48<00:00,  5.19s/it]


In [33]:
for d in tqdm(fps):
    ann = pd.read_csv(list(d.glob('*.csv'))[0], index_col = 0)
    t1, t2 = ann.loc['value', 'start time of acquisition for pass 1'], ann.loc['value', 'start time of acquisition for pass 2']
    t1, t2 = pd.to_datetime(t1).round('d'), pd.to_datetime(t2).round('d')
    if t1.year != 2021: continue
    # heading = ann.loc['value', 'peg heading']
    heading = d.stem.split('_')[1][:3]
    print(heading)

  0%|          | 0/21 [00:00<?, ?it/s]

 19%|█▉        | 4/21 [00:00<00:01, 14.31it/s]

091
091
091
271


 43%|████▎     | 9/21 [00:00<00:00, 17.62it/s]

091
271
091
091


 76%|███████▌  | 16/21 [00:00<00:00, 23.56it/s]

271
091
091
271


100%|██████████| 21/21 [00:01<00:00, 20.43it/s]

091
271
271


In [32]:
d.stem.split('_')[1][:3]

'271'